In [1]:
import numpy as np
import pandas as pd

In [2]:
#最大行列数の調整
pd.get_option('display.max_columns'),pd.get_option('display.max_rows')
pd.set_option('display.max_columns',100)

In [3]:
train_pitch = pd.read_csv('train_pitch.csv')
test_pitch = pd.read_csv('test_pitch.csv')

In [4]:
#label Encoding
#表裏
tb_dic = {'表' : 0, '裏' : 1}
train_pitch['表裏'] = train_pitch['表裏'].map(tb_dic)
test_pitch['表裏'] = test_pitch['表裏'].map(tb_dic)

#右か左か
hands_dic = {'右' : 0, '左' : 1}
train_pitch['打者打席左右'] = train_pitch['打者打席左右'].map(hands_dic)
train_pitch['投手投球左右'] = train_pitch['投手投球左右'].map(hands_dic)
test_pitch['打者打席左右'] = test_pitch['打者打席左右'].map(hands_dic)
test_pitch['投手投球左右'] = test_pitch['投手投球左右'].map(hands_dic)

#先発or救援
roll_dic = {'先発':0, '救援':1}
train_pitch['投手役割'] = train_pitch['投手役割'].map(roll_dic)
test_pitch['投手役割'] = test_pitch['投手役割'].map(roll_dic)

#セ公式、パ公式、セパ交流戦
game_dic = {'セ・リーグ公式戦' :1, 'パ・リーグ公式戦':2, 'セ・パ交流戦':0}
train_pitch['試合種別詳細'] = train_pitch['試合種別詳細'].map(game_dic)
test_pitch['試合種別詳細'] = test_pitch['試合種別詳細'].map(game_dic)

#日付をハイフン無しで
train_pitch['日付'] = train_pitch['日付'].str.replace('-','').astype(int)
test_pitch['日付'] = test_pitch['日付'].str.replace('-','').astype(int)

In [5]:
#塁にいるかいないかにしたい。
def func(x):
    if x == x:
        return 1
    return 0
train_pitch.loc[:,['一塁走者ID','二塁走者ID','三塁走者ID']] = train_pitch.loc[:,['一塁走者ID','二塁走者ID','三塁走者ID']].applymap(func)
test_pitch.loc[:,['一塁走者ID','二塁走者ID','三塁走者ID']] = test_pitch.loc[:,['一塁走者ID','二塁走者ID','三塁走者ID']].applymap(func)

#day or night
def time(x):
    if int(x[0:2]) > 18:
        return 0#day
    else:
        return 1#night
train_pitch['時刻'] = train_pitch['時刻'].map(time)
test_pitch['時刻'] = test_pitch['時刻'].map(time)

#プレイ前走者状況→一旦、居る塁にする。['___', '_2_', '1__', '12_', '_23', '1_3', '__3', '123']
def runner(x):
    if x == '___':
        return 0
    elif x == '1__':
        return 1
    elif x == '_2_':
        return 2
    elif x == '__3':
        return 3
    elif x == '12_':
        return 12
    elif x == '1_3':
        return 13
    elif x == '_23':
        return 23
    else:
        return 123
train_pitch['プレイ前走者状況'] = train_pitch['プレイ前走者状況'].map(runner)
test_pitch['プレイ前走者状況'] = test_pitch['プレイ前走者状況'].map(runner)

In [6]:
train_player = pd.read_csv('train_player.csv')
test_player = pd.read_csv('test_player.csv')

In [7]:
#dicでイジれるやつ
hands_dic['両'] = 2
train_player['投'] = train_player['投'].map(hands_dic)
train_player['打'] = train_player['打'].map(hands_dic)
test_player['投'] = test_player['投'].map(hands_dic)
test_player['打'] = test_player['打'].map(hands_dic)

#大学に行ったか行っていないか
def univ(x):
    if x == '不明':
        return 0
    else:
        return 1
train_player['出身大学名'] = train_player['出身大学名'].map(univ)
test_player['出身大学名'] = test_player['出身大学名'].map(univ)

#出身国→日本or海外
def country(x):
    if x == '日本':
        return 1
    else:
        return 0
train_player['出身国'] = train_player['出身国'].map(country)
test_player['出身国'] = test_player['出身国'].map(country)    

#社会人経験
train_player['社会人'] = train_player['社会人'].map(func)
test_player['社会人'] = test_player['社会人'].map(func)    

#ドラフト種別nan, '高校生', '大学生・社会人', '育成選手'
def draft(x):
    if x == '高校生':
        return 1
    elif x =='大学生・社会人':
        return 2
    elif x =='育成選手':
        return 3
    else:
        return 0
train_player['ドラフト種別'] = train_player['ドラフト種別'].map(draft)
test_player['ドラフト種別'] = test_player['ドラフト種別'].map(draft)


In [8]:
#元のやつにIDベースでくっつけるため年度とIDを足す。
train_pitch['選手ID'] = train_pitch['年度']+train_pitch['投手ID']
train_player['選手ID'] = train_player['年度']+train_player['選手ID']

#元のやつにIDベースでくっつけるため年度とIDを足す。
def merge_word(x,y):
    return str(x)+str(y)
test_pitch['選手ID'] = pd.DataFrame(list(map(merge_word, test_pitch['年度'],test_pitch['投手ID'])))
test_player['選手ID']= pd.DataFrame(list(map(merge_word, test_player['年度'], test_player['選手ID'])))

In [9]:
#merge
train = train_pitch.merge(train_player.drop(['年度'],axis=1), on='選手ID', how='left')
test = test_pitch.merge(test_player.drop(['年度'],axis=1), on='選手ID', how='left')

In [10]:
train.head()

,データ内連番,球種,投球位置区域,年度,試合ID,試合内連番,試合内投球数,日付,時刻,ホームチームID,アウェイチームID,球場ID,球場名,試合種別詳細,イニング,表裏,イニング内打席数,打席内投球数,投手ID,投手チームID,投手投球左右,投手役割,投手登板順,投手試合内対戦打者数,投手試合内投球数,投手イニング内投球数,打者ID,打者チームID,打者打席左右,打者打順,打者守備位置,打者試合内打席数,プレイ前ホームチーム得点数,プレイ前アウェイチーム得点数,プレイ前アウト数,プレイ前ボール数,プレイ前ストライク数,プレイ前走者状況,一塁走者ID,二塁走者ID,三塁走者ID,捕手ID,一塁手ID,二塁手ID,三塁手ID,遊撃手ID,左翼手ID,中堅手ID,右翼手ID,成績対象投手ID,成績対象打者ID,選手ID,チームID,チーム名,選手名,育成選手F,背番号,位置,投,打,身長,体重,生年月日,出身高校ID,出身高校名,出身大学ID,出身大学名,社会人,ドラフト年,ドラフト種別,ドラフト順位,年俸,出身国,出身地,血液型
0,0,0,0,2017,2017033101,1,1,20170331,1,1,4,1,東京ドーム,1,1,0,1,1,1500001,1,0,0,1,1,1,1,900410,4,1,1,中,1,0,0,0,0,0,0,0,0,0,1300027,12049,800002,1200136,700003,1400101,900093,900397,1500001,900410,1502018,1,巨人,マイコラス,0,39,投手,0,0,196,100,1988/8/23,0,不明,0,0,0,NaN,0,NaN,24000,0,不明,不明
1,1,0,8,2017,2017033101,2,2,20170331,1,1,4,1,東京ドーム,1,1,0,1,2,1500001,1,0,0,1,1,2,2,900410,4,1,1,中,1,0,0,0,0,1,0,0,0,0,1300027,12049,800002,1200136,700003,1400101,900093,900397,1500001,900410,1502018,1,巨人,マイコラス,0,39,投手,0,0,196,100,1988/8/23,0,不明,0,0,0,NaN,0,NaN,24000,0,不明,不明
2,2,0,5,2017,2017033101,3,3,20170331,1,1,4,1,東京ドーム,1,1,0,1,3,1500001,1,0,0,1,1,3,3,900410,4,1,1,中,1,0,0,0,0,2,0,0,0,0,1300027,12049,800002,1200136,700003,1400101,900093,900397,1500001,900410,1502018,1,巨人,マイコラス,0,39,投手,0,0,196,100,1988/8/23,0,不明,0,0,0,NaN,0,NaN,24000,0,不明,不明
3,3,0,12,2017,2017033101,4,4,20170331,1,1,4,1,東京ドーム,1,1,0,2,1,1500001,1,0,0,1,2,4,4,11436,4,0,2,二,1,0,0,1,0,0,0,0,0,0,1300027,12049,800002,1200136,700003,1400101,900093,900397,1500001,11436,1502018,1,巨人,マイコラス,0,39,投手,0,0,196,100,1988/8/23,0,不明,0,0,0,NaN,0,NaN,24000,0,不明,不明
4,4,0,8,2017,2017033101,5,5,20170331,1,1,4,1,東京ドーム,1,1,0,2,2,1500001,1,0,0,1,2,5,5,11436,4,0,2,二,1,0,0,1,0,1,0,0,0,0,1300027,12049,800002,1200136,700003,1400101,900093,900397,1500001,11436,1502018,1,巨人,マイコラス,0,39,投手,0,0,196,100,1988/8/23,0,不明,0,0,0,NaN,0,NaN,24000,0,不明,不明


In [11]:
train.columns

Index(['データ内連番', '球種', '投球位置区域', '年度', '試合ID', '試合内連番', '試合内投球数', '日付', '時刻',
       'ホームチームID', 'アウェイチームID', '球場ID', '球場名', '試合種別詳細', 'イニング', '表裏',
       'イニング内打席数', '打席内投球数', '投手ID', '投手チームID', '投手投球左右', '投手役割', '投手登板順',
       '投手試合内対戦打者数', '投手試合内投球数', '投手イニング内投球数', '打者ID', '打者チームID', '打者打席左右',
       '打者打順', '打者守備位置', '打者試合内打席数', 'プレイ前ホームチーム得点数', 'プレイ前アウェイチーム得点数',
       'プレイ前アウト数', 'プレイ前ボール数', 'プレイ前ストライク数', 'プレイ前走者状況', '一塁走者ID', '二塁走者ID',
       '三塁走者ID', '捕手ID', '一塁手ID', '二塁手ID', '三塁手ID', '遊撃手ID', '左翼手ID', '中堅手ID',
       '右翼手ID', '成績対象投手ID', '成績対象打者ID', '選手ID', 'チームID', 'チーム名', '選手名',
       '育成選手F', '背番号', '位置', '投', '打', '身長', '体重', '生年月日', '出身高校ID', '出身高校名',
       '出身大学ID', '出身大学名', '社会人', 'ドラフト年', 'ドラフト種別', 'ドラフト順位', '年俸', '出身国',
       '出身地', '血液型'],
      dtype='object')

In [34]:
tr = train.drop(['球場名', '打者守備位置','チーム名', '出身高校名','位置'],axis=1)

tr

,データ内連番,球種,投球位置区域,年度,試合ID,試合内連番,試合内投球数,日付,時刻,ホームチームID,アウェイチームID,球場ID,試合種別詳細,イニング,表裏,イニング内打席数,打席内投球数,投手ID,投手チームID,投手投球左右,投手役割,投手登板順,投手試合内対戦打者数,投手試合内投球数,投手イニング内投球数,打者ID,打者チームID,打者打席左右,打者打順,打者試合内打席数,プレイ前ホームチーム得点数,プレイ前アウェイチーム得点数,プレイ前アウト数,プレイ前ボール数,プレイ前ストライク数,プレイ前走者状況,一塁走者ID,二塁走者ID,三塁走者ID,捕手ID,一塁手ID,二塁手ID,三塁手ID,遊撃手ID,左翼手ID,中堅手ID,右翼手ID,成績対象投手ID,成績対象打者ID,選手ID,チームID,選手名,育成選手F,背番号,投,打,身長,体重,生年月日,出身高校ID,出身大学ID,出身大学名,社会人,ドラフト年,ドラフト種別,ドラフト順位,年俸,出身国,出身地,血液型
0,0,0,0,2017,2017033101,1,1,20170331,1,1,4,1,1,1,0,1,1,1500001,1,0,0,1,1,1,1,900410,4,1,1,1,0,0,0,0,0,0,0,0,0,1300027,12049,800002,1200136,700003,1400101,900093,900397,1500001,900410,1502018,1,マイコラス,0,39,0,0,196,100,1988/8/23,0,0,0,0,NaN,0,NaN,24000,0,不明,不明
1,1,0,8,2017,2017033101,2,2,20170331,1,1,4,1,1,1,0,1,2,1500001,1,0,0,1,1,2,2,900410,4,1,1,1,0,0,0,0,1,0,0,0,0,1300027,12049,800002,1200136,700003,1400101,900093,900397,1500001,900410,1502018,1,マイコラス,0,39,0,0,196,100,1988/8/23,0,0,0,0,NaN,0,NaN,24000,0,不明,不明
2,2,0,5,2017,2017033101,3,3,20170331,1,1,4,1,1,1,0,1,3,1500001,1,0,0,1,1,3,3,900410,4,1,1,1,0,0,0,0,2,0,0,0,0,1300027,12049,800002,1200136,700003,1400101,900093,900397,1500001,900410,1502018,1,マイコラス,0,39,0,0,196,100,1988/8/23,0,0,0,0,NaN,0,NaN,24000,0,不明,不明
3,3,0,12,2017,2017033101,4,4,20170331,1,1,4,1,1,1,0,2,1,1500001,1,0,0,1,2,4,4,11436,4,0,2,1,0,0,1,0,0,0,0,0,0,1300027,12049,800002,1200136,700003,1400101,900093,900397,1500001,11436,1502018,1,マイコラス,0,39,0,0,196,100,1988/8/23,0,0,0,0,NaN,0,NaN,24000,0,不明,不明
4,4,0,8,2017,2017033101,5,5,20170331,1,1,4,1,1,1,0,2,2,1500001,1,0,0,1,2,5,5,11436,4,0,2,1,0,0,1,0,1,0,0,0,0,1300027,12049,800002,1200136,700003,1400101,900093,900397,1500001,11436,1502018,1,マイコラス,0,39,0,0,196,100,1988/8/23,0,0,0,0,NaN,0,NaN,24000,0,不明,不明
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257112,257112,0,12,2017,2017101002,293,288,20171010,0,5,4,5,1,9,0,4,1,1600115,5,0,1,6,7,20,13,1400160,4,0,7,1,6,1,2,0,0,1,1,0,0,500066,1000145,1600113,600080,1400172,1000001,1500086,1400170,1600115,1400160,1602132,5,才木 浩人,0,35,0,0,188,79,1998/11/7,32590,0,0,0,2016.0,0,3.0,600,1,兵庫,A
257113,257113,0,10,2017,2017101002,294,289,20171010,0,5,4,5,1,9,0,4,2,1600115,5,0,1,6,7,21,14,1400160,4,0,7,1,6,1,2,0,1,1,1,0,0,500066,1000145,1600113,600080,1400172,1000001,1500086,1400170,1600115,1400160,1602132,5,才木 浩人,0,35,0,0,188,79,1998/11/7,32590,0,0,0,2016.0,0,3.0,600,1,兵庫,A
257114,257114,0,1,2017,2017101002,295,290,20171010,0,5,4,5,1,9,0,4,3,1600115,5,0,1,6,7,22,15,1400160,4,0,7,1,6,1,2,1,1,1,1,0,0,500066,1000145,1600113,600080,1400172,1000001,1500086,1400170,1600115,1400160,1602132,5,才木 浩人,0,35,0,0,188,79,1998/11/7,32590,0,0,0,2016.0,0,3.0,600,1,兵庫,A
257115,257115,0,9,2017,2017101002,296,291,20171010,0,5,4,5,1,9,0,4,4,1600115,5,0,1,6,7,23,16,1400160,4,0,7,1,6,1,2,1,2,1,1,0,0,500066,1000145,1600113,600080,1400172,1000001,1500086,1400170,1600115,1400160,1602132,5,才木 浩人,0,35,0,0,188,79,1998/11/7,32590,0,0,0,2016.0,0,3.0,600,1,兵庫,A


In [35]:
tr_groupby_balls = tr.groupby(['選手名','球種']).mean()
tr_groupby_area = tr.groupby(['選手名','投球位置区域']).mean()
tr_groupby = tr.groupby(['選手名','投球位置区域','球種']).mean()

In [38]:
tr_groupby[:50]

データ内連番      年度          試合ID       試合内連番      試合内投球数  \
選手名    投球位置区域 球種                                                                
アラウホ   0      0   146389.714286  2017.0  2.017072e+09  203.285714  198.142857   
              2    61147.000000  2017.0  2.017051e+09  298.000000  287.000000   
       1      0   159305.625000  2017.0  2.017075e+09  239.875000  232.250000   
              2   113823.000000  2017.0  2.017064e+09  266.333333  258.000000   
       2      0   152106.700000  2017.0  2.017074e+09  198.900000  193.800000   
              2    64563.000000  2017.0  2.017052e+09  197.000000  190.000000   
       3      0   134604.000000  2017.0  2.017069e+09  222.750000  216.250000   
              2    64616.000000  2017.0  2.017052e+09  250.000000  243.000000   
       4      0    95697.777778  2017.0  2.017060e+09  265.444444  256.555556   
              2   108540.666667  2017.0  2.017064e+09  237.000000  230.666667   
       5      0   134742.500000  2017.0  2.017071e+09  213.500000  208.000000   
              2    64579.000000  2017.0  2.017052e+09  213.000000  206.000000   
       6      0   173413.000000  2017.0  2.017077e+09  211.750000  206.750000   
              2    64569.000000  2017.0  2.017052e+09  203.000000  196.000000   
              5   138992.500000  2017.0  2.017071e+09  207.000000  202.000000   
       7      0   152161.200000  2017.0  2.017075e+09  194.200000  189.600000   
              5   175105.250000  2017.0  2.017079e+09  261.500000  253.750000   
       8      0   210594.000000  2017.0  2.017087e+09  183.000000  179.000000   
              5   137260.000000  2017.0  2.017071e+09  233.000000  226.000000   
       9      0   117361.727273  2017.0  2.017065e+09  250.181818  242.363636   
       10     0   160902.739130  2017.0  2.017076e+09  221.391304  215.000000   
              2   223076.000000  2017.0  2.017091e+09  376.000000  361.000000   
              3   207827.000000  2017.0  2.017083e+09  210.000000  205.000000   
              5   167017.666667  2017.0  2.017078e+09  260.666667  252.333333   
       11     0   148843.142857  2017.0  2.017074e+09  215.285714  209.428571   
              2    97048.000000  2017.0  2.017060e+09  235.142857  227.857143   
              5   211986.500000  2017.0  2.017088e+09  178.250000  175.000000   
       12     0   113806.500000  2017.0  2.017064e+09  250.000000  241.500000   
              2   134300.500000  2017.0  2.017068e+09  215.166667  209.333333   
              3   207828.000000  2017.0  2.017083e+09  211.000000  206.000000   
              5   183761.800000  2017.0  2.017081e+09  246.600000  239.200000   
ウィーランド 0      0   128383.082192  2017.0  2.017067e+09   83.479452   81.082192   
              1   161683.352941  2017.0  2.017075e+09  123.352941  120.705882   
              3   188434.700000  2017.0  2.017082e+09  124.000000  121.000000   
              5    96166.636364  2017.0  2.017060e+09   96.363636   93.363636   
       1      0   144968.803922  2017.0  2.017071e+09  115.627451  112.362745   
              1   162397.592593  2017.0  2.017075e+09  158.666667  155.444444   
              3   177564.666667  2017.0  2.017077e+09  137.000000  133.833333   
              5   118852.909091  2017.0  2.017065e+09  126.500000  123.000000   
       2      0   137304.376623  2017.0  2.017069e+09  106.831169  104.662338   
              1   161010.296296  2017.0  2.017075e+09  112.370370  110.407407   
              3   150749.375000  2017.0  2.017073e+09  121.625000  118.250000   
              5   159689.263158  2017.0  2.017075e+09  138.368421  135.947368   
       3      0   134823.455882  2017.0  2.017069e+09  102.676471  100.294118   
              1   156391.240000  2017.0  2.017074e+09  119.400000  116.360000   
              3   107607.000000  2017.0  2.017059e+09  177.666667  171.666667   
              5   130785.187500  2017.0  2.017067e+09  113.875000  108.937500   
       4      0   157036.987654  2017.0  2.017075e+09  1